In [1]:
import math 
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
from utils import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
''' 
    Read one week of data 
'''
week = 1

# Format the week with leading zeros
number = f"{week:02d}"

# Path of file
base_path = f"data/X{number}/X{number}"

# Load file
data = np.loadtxt(base_path)

# Keep first 144 entries from each line 
data = data[:, :144]

In [3]:
# Normalize the data
scaler = MinMaxScaler()
data_norm = normalize_matrix(scaler, data)

# Train-Test Split 
train_data, test_data = train_test_split(data_norm, 0.8)

# Window the dataset
trainX, trainY = create_dataset(train_data, 10)
testX, testY = create_dataset(test_data, 10)

In [4]:
# Specify model parameters 
input_size = trainX.shape[2] # Number of features in input
hidden_size = 200  # Number of features in hidden state
output_size = 144  # Number of output classes 
learn_rate = 0.001 
epochs = 100
num_layers = 1
batch_size = 32
shuffle = False #don't want to lose the time dependency
num_workers = 4  # Number of subprocesses to use for data loading

In [5]:
# create torch datasets and torch dataloader

train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(trainX),
                                                 torch.Tensor(trainY))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             num_workers=num_workers,
                                             shuffle=shuffle)

test_dataset  = torch.utils.data.TensorDataset(torch.FloatTensor(testX),
                                                 torch.Tensor(testY))


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                             num_workers=num_workers,
                                             shuffle=shuffle)


In [28]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.rnn = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.sigmoid = nn.Sigmoid()
        self.out = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, _ = self.rnn(x, None)  # None represents zero initial hidden state
        out = self.out(r_out[:, -1, :])  # return the last value
        out = self.sigmoid(out)
        return out
    
import time

# Define training function
def train(model, train_loader, epochs, criterion, optimizer):
    ''' Train ML Model''' 

    print_interval = 5
    track_losses = np.zeros(epochs)
    start = time.time()

    for epoch in range(epochs): 
        for inputs, targets in train_loader: 
            
            # Pass data to LSTM
            optimizer.zero_grad()
            outputs = model(inputs) #size ->: batch_size x output_size
            
            # Compute the loss
            loss = criterion(outputs, targets) #size ->: float
            
            
            # Compute the gradient and update the network parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        training_loss = loss.item()
        track_losses[epoch] = training_loss 

        if (epoch) % (print_interval-1) == 0:
            print('epoch: %4d training loss:%10.3e time:%7.1f'%(epoch, training_loss, time.time()-start))

    return loss   

The naive solution uses the output of the MLU and the ground truth MLU, computes the MLU, and performs back propogation

In [29]:
class NaiveLoss(nn.Module): 

    def __init__(self):
        super(NaiveLoss, self).__init__()

    def forward(self, inputs, targets):
        G = abilene_topo_lossfn()

        u_gt = torch.empty((1, targets.shape[0]))
        u_pred = torch.empty((1, targets.shape[0]))
        
        for i in range(inputs.shape[0]):
            input = inputs[i, :].reshape((12, 12))
            target = targets[i, :].reshape((12, 12))

            target.fill_diagonal_(0)
            input.fill_diagonal_(0)
            
            u = MinMaxLinkUtil(G, target)
            u_prime = MinMaxLinkUtil(G, input)

            u_gt[0, i] = u
            u_pred[0, i] = u_prime
            
        u_gt.requires_grad_()
        u_pred.requires_grad_()

        comp_loss= nn.MSELoss()
        return comp_loss(u_gt, u_pred)

In [30]:
# Create Model 
model = RNN(input_size, hidden_size, num_layers)

# Create optimizer 
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

# Create loss function
criterion = NaiveLoss()

loss = train(model, train_loader, epochs, criterion, optimizer)

epoch:    0 training loss: 3.670e-01 time:  451.5


KeyboardInterrupt: 